In [1]:
#!/usr/bin/env python
"""
Training GNN
"""

import tensorflow as tf
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

import os
import sys
prog_name = os.path.basename(sys.argv[0])

In [20]:
import argparse
import glob
import re
import time

import numpy as np
import sklearn.metrics
import sonnet as snt

from graph_nets import utils_tf
from graph_nets import utils_np

from heptrkx.nx_graph import utils_train
from heptrkx.nx_graph import prepare
from heptrkx.nx_graph import get_model
from heptrkx import load_yaml
from heptrkx.nx_graph.utils_io import ckpt_name

# parser = argparse.ArgumentParser(description='Train nx-graph with configurations')
# add_arg = parser.add_argument
# add_arg('config',  nargs='?', default='configs/data_5000evts.yaml')

# args = parser.parse_args()

all_config = load_yaml('../configs/train_edge_classifier_kaggle_share.yaml')
config = all_config['segment_training']


# add ops to save and restore all the variables
prod_name = config['prod_name']
output_dir = os.path.join(config['output_dir'], prod_name)
print("[{}] save models at {}".format(prog_name, output_dir))
os.makedirs(output_dir, exist_ok=True)

files = glob.glob(output_dir+"/*.ckpt.meta")
last_iteration = 0 if len(files) < 1 else max([
    int(re.search('checkpoint_([0-9]*).ckpt.meta', os.path.basename(x)).group(1))
    for x in files
])
print(files)
# print("jajdsjfsaifjsaof")
# print(len(files))
print("[{}] last iteration: {}".format(prog_name, last_iteration))

# default 2/3 for training and 1/3 for testing
input_nxgraphs_dir = all_config['make_graph']['out_graph']
generate_input_target = prepare.inputs_generator(input_nxgraphs_dir, n_train_fraction=0.8)

config_tr = config['parameters']
# How much time between logging and printing the current results.
# save checkpoint very 10 mins
log_every_seconds       = config_tr['time_lapse']
batch_size = n_graphs   = config_tr['batch_size']   # need optimization
num_training_iterations = config_tr['iterations']
iter_per_job            = 2500 if 'iter_per_job' not in config_tr else config_tr['iter_per_job']
num_processing_steps_tr = config_tr['n_iters']      ## level of message-passing
print("Maximum iterations per job: {}".format(iter_per_job))

[ipykernel_launcher.py] save models at ../out/segments_100/v0_kaggle
[]
[ipykernel_launcher.py] last iteration: 0
Total Events: 100 with 16 sections, total 1600 files 
Training data: [0, 79] events, total 1280 files
Testing data:  [80, 100] events, total 320 files
Training and testing graphs are selected sequantially from their corresponding pools
Maximum iterations per job: 80000


Get data
---------

In [21]:
#TODO:
@tf.function
def get_data(n_graphs, is_trained=True):
    inputs, targets = generate_input_target(n_graphs, is_trained)
    if isinstance(inputs[0], dict):
        input_graphs  = utils_np.data_dicts_to_graphs_tuple(inputs)
        target_graphs = utils_np.data_dicts_to_graphs_tuple(targets)
    else:
        input_graphs  = utils_np.networkxs_to_graphs_tuple(inputs)
        target_graphs = utils_np.networkxs_to_graphs_tuple(targets)
    return input_graphs, target_graphs

Initialize input_graphs and target_graphs as instances of GraphsTuple
---------------

In [22]:
model = get_model(config['model_name'])
input_graphs, target_graphs = get_data(n_graphs)
# dir(model)

Optimizer
------------

In [23]:
global_step = tf.Variable(0, trainable=False)
start_learning_rate = config_tr['learning_rate'] # 0.001
learning_rate = tf.compat.v1.train.exponential_decay(
    start_learning_rate, global_step,
    decay_steps=500,
    decay_rate=0.97, staircase=True)

# using a constant learning rate instead:
# snt Adam doesn't seem to support decaying learning rate
# use tf.keras.optimizers.adam for decaying learning rate
optimizer = snt.optimizers.Adam(1e-3)

loss_weights matrix
------------------

In [24]:
loss_weights = 1.0
if config_tr['real_weight']:
    real_weight = config_tr['real_weight']
    fake_weight = config_tr['fake_weight']
    loss_weights = target_graphs.edges * real_weight + (1 - target_graphs.edges)*fake_weight

Training step:
-----------------

In [25]:
def update_step(inputs_tr, targets_tr, loss_weights):
    with tf.GradientTape() as tape:
        outputs_tr = model._build(inputs_tr, num_processing_steps_tr)
#         print(outputs_tr)
#         print('ccc')
        # Loss:
        losses_tr = utils_train.create_loss_ops(targets_tr, outputs_tr, loss_weights)
        loss_tr = sum(losses_tr) / num_processing_steps_tr
    gradients = tape.gradient(loss_tr, model.trainable_variables)
#     print('aaa')
    optimizer.apply(gradients, model.trainable_variables)
#     print('ddd')
    return outputs_tr, loss_tr

Compiling update_step with tf_function to speed up code:
-------------------

In [26]:
example_input_data, example_target_data = get_data(n_graphs)
input_signature = [
    utils_tf.specs_from_graphs_tuple(example_input_data),
    utils_tf.specs_from_graphs_tuple(example_target_data),
    tf.TensorSpec.from_tensor(loss_weights)
]

# # use this function instead of update_step:
compiled_update_step = tf.function(update_step, input_signature=input_signature)

Restore Previous Checkpoint
-------

In [27]:
# both optimizer and model are snt version, they should be trackable
ckpt = tf.train.Checkpoint(step=tf.Variable(1), optimizer=optimizer, model=model)
manager = tf.train.CheckpointManager(ckpt, os.path.join(output_dir, ckpt_name.format(last_iteration)), max_to_keep=3)

ckpt.restore(manager.latest_checkpoint)
if manager.latest_checkpoint:
    print("loading checkpoint:", os.path.join(output_dir, ckpt_name.format(last_iteration)))
else:
    print("Initializing from scratch.")

loading checkpoint: ../out/segments_100/v0_kaggle/checkpoint_00000.ckpt


Initialize
--------

In [29]:
logged_iterations = []
losses_tr = []
corrects_tr = []
solveds_tr = []

Log Training Process
-----------

In [30]:
out_str  = time.strftime('%d %b %Y %H:%M:%S', time.localtime())
out_str += '\n'
out_str += "# (iteration number), T (elapsed seconds), Ltr (training loss), Precision, Recall\n"
log_name = os.path.join(output_dir, config['log_name'])
with open(log_name, 'a') as f:
    f.write(out_str)




In [31]:
# instead of using feed_dict, manually take the values out
inputs_tr, targets_tr = get_data(batch_size)
# print(inputs_tr)
outputs_tr, loss_tr = update_step(inputs_tr, targets_tr, loss_weights)


Run Training Steps:
---------------

In [32]:
start_time = time.time()
last_log_time = start_time

## loop over iterations, each iteration generating a batch of data for training
iruns = 0
all_run_time = start_time
all_data_taking_time = start_time

print("# (iteration number), TD (get graph), TR (TF run)")
for iteration in range(last_iteration, num_training_iterations):
    if iruns > iter_per_job:
        print("runs larger than {} iterations per job, stop".format(iter_per_job))
        break
    else: iruns += 1
    last_iteration = iteration
    data_start_time = time.time()
    
    
    # instead of using feed_dict, manually take the values out
    inputs_tr, targets_tr = get_data(batch_size)
    loss_weights = targets_tr.edges * real_weight + (1 - targets_tr.edges)*fake_weight
    all_data_taking_time += time.time() - data_start_time

    # timing the run time only 
    run_start_time = time.time()

    # added this: not sure if correct
#     print('aaa')
    outputs_tr, loss_tr = update_step(inputs_tr, targets_tr, loss_weights)
#     print('bbb')
    run_time = time.time() - run_start_time
    all_run_time += run_time

    the_time = time.time()
    elapsed_since_last_log = the_time - last_log_time

    if elapsed_since_last_log > log_every_seconds:
        # save a checkpoint
#         print('in here')
        last_log_time = the_time
        
        inputs_ge, targets_ge = get_data(batch_size, is_trained=False)
        loss_weights = targets_ge.edges * real_weight + (1 - targets_ge.edges)*fake_weight
        
        outputs_ge = model._build(inputs_ge, num_processing_steps_tr)

        losses_ge = utils_train.create_loss_ops(targets_ge, outputs_ge, loss_weights)
        loss_ge = sum(losses_ge) / num_processing_steps_tr
        


        correct_tr, solved_tr = utils_train.compute_matrics(
            targets_ge, outputs_ge[-1])
        elapsed = time.time() - start_time
        losses_tr.append(loss_tr)
        corrects_tr.append(correct_tr)
        solveds_tr.append(solved_tr)
        logged_iterations.append(iteration)
        out_str = "# {:05d}, T {:.1f}, Ltr {:.4f}, Lge {:.4f}, Precision {:.4f}, Recall {:.4f}\n".format(
            iteration, elapsed, loss_tr, loss_ge,
            correct_tr, solved_tr)

        run_cost_time = all_run_time - start_time
        data_cost_time = all_data_taking_time - start_time
        print("# {:05d}, TD {:.1f}, TR {:.1f}".format(iteration, data_cost_time, run_cost_time))
        with open(log_name, 'a') as f:
            f.write(out_str)
        
        ckpt.step.assign_add(1)
        save_path = manager.save()

# (iteration number), TD (get graph), TR (TF run)


/global/homes/c/cheliu/.conda/envs/upgrade/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# 00003, TD 0.0, TR 61.9


/global/homes/c/cheliu/.conda/envs/upgrade/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# 00007, TD 0.0, TR 121.0


...

/global/homes/c/cheliu/.conda/envs/upgrade/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# 00043, TD 0.1, TR 668.0


/global/homes/c/cheliu/.conda/envs/upgrade/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# 00047, TD 0.1, TR 727.6


KeyboardInterrupt: 